In [1]:
import pandas as pd
import os

In [2]:
# Import 2019 lacity data.
dir = pwd
dfweb = pd.read_csv(os.path.join(dir, 'MyLA311_Service_Request_Data_2019.csv'), low_memory=False)

In [3]:
# Import 2019 api call data from the pipeline.
dfcall = pd.read_csv(os.path.join(dir, 'clean_311_data_2019.csv'), low_memory=False, index_col=0)

In [4]:
# Import 2019 crime data.
dfcrime = pd.read_csv(os.path.join(dir, 'crime/cr2019b.csv'), low_memory=False)

In [5]:
dfcrime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213315 entries, 0 to 213314
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   DATE OCC     213315 non-null  object
 1   TIME OCC     213315 non-null  int64 
 2   AreaName     213315 non-null  object
 3   Rpt Dist No  213315 non-null  int64 
 4   Crm Cd       213315 non-null  int64 
 5   Crm Cd Desc  213315 non-null  object
 6   Status Desc  213315 non-null  object
 7   LOCATION     213315 non-null  object
dtypes: int64(3), object(5)
memory usage: 13.0+ MB


In [6]:
dfcall.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1306791 entries, 0 to 1306790
Data columns (total 15 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   requestId    1306791 non-null  int64  
 1   srnumber     1306791 non-null  object 
 2   councilId    1306791 non-null  int64  
 3   councilName  1306791 non-null  object 
 4   typeId       1306791 non-null  int64  
 5   typeName     1306791 non-null  object 
 6   agencyId     1306791 non-null  int64  
 7   agencyName   1306791 non-null  object 
 8   sourceId     1306791 non-null  int64  
 9   sourceName   1306791 non-null  object 
 10  createdDate  1306791 non-null  object 
 11  closedDate   1305901 non-null  object 
 12  address      1306791 non-null  object 
 13  latitude     1306791 non-null  float64
 14  longitude    1306791 non-null  float64
dtypes: float64(2), int64(5), object(8)
memory usage: 159.5+ MB


In [7]:
dfweb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1308093 entries, 0 to 1308092
Data columns (total 34 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   SRNumber                   1308093 non-null  object 
 1   CreatedDate                1308093 non-null  object 
 2   UpdatedDate                1308093 non-null  object 
 3   ActionTaken                1308093 non-null  object 
 4   Owner                      1308093 non-null  object 
 5   RequestType                1308093 non-null  object 
 6   Status                     1308093 non-null  object 
 7   RequestSource              1308093 non-null  object 
 8   CreatedByUserOrganization  1308093 non-null  object 
 9   MobileOS                   320743 non-null   object 
 10  Anonymous                  1308093 non-null  object 
 11  AssignTo                   1288424 non-null  object 
 12  ServiceDate                1231245 non-null  object 
 13  ClosedDate  

In [8]:
# Renaming the merging column.
dfweb.rename(columns = {'SRNumber': 'srnumber'}, inplace = True)

In [9]:
# Merging the 311 request data from the 311 server and the lacity.org on the column srnumber.
df311 = pd.merge(dfcall, dfweb, how = 'left', on = 'srnumber')

In [10]:
# Renaming the PolicePrecinct column to AreaName.
df311.rename(columns = {'PolicePrecinct': 'AreaName'}, inplace = True)

In [11]:
df311_subset = df311[['srnumber' , 'councilId', 'councilName', 'typeName', 'agencyName', 'sourceName', 'createdDate', 'closedDate', 'address', 'latitude', 'longitude', 'AreaName' ]]

In [12]:
df311_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1306791 entries, 0 to 1306790
Data columns (total 12 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   srnumber     1306791 non-null  object 
 1   councilId    1306791 non-null  int64  
 2   councilName  1306791 non-null  object 
 3   typeName     1306791 non-null  object 
 4   agencyName   1306791 non-null  object 
 5   sourceName   1306791 non-null  object 
 6   createdDate  1306791 non-null  object 
 7   closedDate   1305901 non-null  object 
 8   address      1306791 non-null  object 
 9   latitude     1306791 non-null  float64
 10  longitude    1306791 non-null  float64
 11  AreaName     1306771 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 129.6+ MB


In [13]:
df311_subset.typeName.unique()

array(['Bulky Items', 'Water Waste', 'Graffiti', 'Illegal Dumping',
       'Single Streetlight', 'Homeless Encampment',
       'Multiple Streetlights', 'Other', 'Animal Remains',
       'Electronic Waste', 'Metal/Appliances', 'Feedback'], dtype=object)

In [14]:
dfcrime.AreaName.head()

0    Wilshire
1     Mission
2     Pacific
3     Rampart
4     Mission
Name: AreaName, dtype: object

In [15]:
df311_subset.AreaName.head()

0             OLYMPIC
1    WEST LOS ANGELES
2          DEVONSHIRE
3          DEVONSHIRE
4          DEVONSHIRE
Name: AreaName, dtype: object

In [16]:
dfcrime['AreaName'] = dfcrime['AreaName'].apply(str.upper)

In [17]:
# Merging the crime data with the 311 data
df2019 = pd.merge(df311_subset, dfcrime, on = 'AreaName', how = 'left', indicator = True)

MemoryError: Unable to allocate 88.4 GiB for an array with shape (11862294628,) and data type int64

### Going to check for 0 values, duplicates, NAN values in the merging column.

In [18]:
# Checking for 0 values in the merging column.
len(df311_subset[df311_subset['AreaName']== 0])

0

In [19]:
len(dfcrime[dfcrime['AreaName'] == 0])

0

In [20]:
# Checking for NAN values.
dfcrime.isna().sum()

DATE OCC       0
TIME OCC       0
AreaName       0
Rpt Dist No    0
Crm Cd         0
Crm Cd Desc    0
Status Desc    0
LOCATION       0
dtype: int64

In [21]:
df311_subset.isna().sum()

srnumber         0
councilId        0
councilName      0
typeName         0
agencyName       0
sourceName       0
createdDate      0
closedDate     890
address          0
latitude         0
longitude        0
AreaName        20
dtype: int64

In [22]:
df311_subset = df311_subset.dropna()

In [23]:
df311_subset.isna().sum()

srnumber       0
councilId      0
councilName    0
typeName       0
agencyName     0
sourceName     0
createdDate    0
closedDate     0
address        0
latitude       0
longitude      0
AreaName       0
dtype: int64

In [24]:
dfcrime.duplicated().sum()

3

In [25]:
# Remove the duplicated values...
dfcrime.drop_duplicates(inplace = True)

In [26]:
dfcrime.duplicated().sum()

0

In [27]:
df311_subset.duplicated().sum()

121

In [28]:
# Remove the duplicated values.
df311_subset = df311_subset.drop_duplicates()

In [29]:
df311_subset.duplicated().sum()

0

In [30]:
# Changing the dtype.
dfcrime['TIME OCC'] = dfcrime['TIME OCC'].astype(float)
dfcrime['Rpt Dist No'] = dfcrime['Rpt Dist No'].astype(float)
dfcrime['Crm Cd'] = dfcrime['Crm Cd'].astype(float)
df311_subset['councilId'] = df311_subset['councilId'].astype(float)

In [31]:
# Merging the crime data with the 311 data
df2019 = pd.merge(df311_subset, dfcrime, on = 'AreaName', how = 'left', indicator = True)

MemoryError: Unable to allocate 88.3 GiB for an array with shape (11855468327,) and data type int64

### Still get the memory error.

In [32]:
# Okay going to take a subset of data from both the frames.
df311_1000 = df311_subset.head(1000)

In [33]:
df311_1000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   srnumber     1000 non-null   object 
 1   councilId    1000 non-null   float64
 2   councilName  1000 non-null   object 
 3   typeName     1000 non-null   object 
 4   agencyName   1000 non-null   object 
 5   sourceName   1000 non-null   object 
 6   createdDate  1000 non-null   object 
 7   closedDate   1000 non-null   object 
 8   address      1000 non-null   object 
 9   latitude     1000 non-null   float64
 10  longitude    1000 non-null   float64
 11  AreaName     1000 non-null   object 
dtypes: float64(3), object(9)
memory usage: 101.6+ KB


In [34]:
dfcrime_1000 = dfcrime.head(1000)

In [35]:
df2019_1000 = pd.merge(df311_1000, dfcrime_1000, on = 'AreaName', how = 'inner', indicator = True)

In [36]:
df2019_1000.head()

,srnumber,councilId,councilName,typeName,agencyName,sourceName,createdDate,closedDate,address,latitude,longitude,AreaName,DATE OCC,TIME OCC,Rpt Dist No,Crm Cd,Crm Cd Desc,Status Desc,LOCATION,_merge
0,1-1262692791,32.0,Greater Wilshire,Bulky Items,Sanitation Bureau,Website,2019-01-01T00:02:00,2019-01-04T11:03:00,"616 N GRAMERCY PL, 90004",34.082258,-118.312461,OLYMPIC,7/16/2019,1500.0,2014.0,354.0,THEFT OF IDENTITY,Invest Cont,200 S HARVARD BL,both
1,1-1262692791,32.0,Greater Wilshire,Bulky Items,Sanitation Bureau,Website,2019-01-01T00:02:00,2019-01-04T11:03:00,"616 N GRAMERCY PL, 90004",34.082258,-118.312461,OLYMPIC,11/21/2019,1745.0,2024.0,440.0,THEFT PLAIN - PETTY ($950 & UNDER),Invest Cont,WILSHIRE BL,both
2,1-1262692791,32.0,Greater Wilshire,Bulky Items,Sanitation Bureau,Website,2019-01-01T00:02:00,2019-01-04T11:03:00,"616 N GRAMERCY PL, 90004",34.082258,-118.312461,OLYMPIC,9/18/2019,2200.0,2076.0,624.0,BATTERY - SIMPLE ASSAULT,Invest Cont,1500 S VERMONT AV,both
3,1-1262692791,32.0,Greater Wilshire,Bulky Items,Sanitation Bureau,Website,2019-01-01T00:02:00,2019-01-04T11:03:00,"616 N GRAMERCY PL, 90004",34.082258,-118.312461,OLYMPIC,2/22/2019,2310.0,2002.0,230.0,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",Invest Cont,WESTERN,both
4,1-1262692791,32.0,Greater Wilshire,Bulky Items,Sanitation Bureau,Website,2019-01-01T00:02:00,2019-01-04T11:03:00,"616 N GRAMERCY PL, 90004",34.082258,-118.312461,OLYMPIC,9/7/2019,900.0,2021.0,310.0,BURGLARY,Invest Cont,600 S GRAMERCY PL,both


In [37]:
df2019_1000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41063 entries, 0 to 41062
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   srnumber     41063 non-null  object  
 1   councilId    41063 non-null  float64 
 2   councilName  41063 non-null  object  
 3   typeName     41063 non-null  object  
 4   agencyName   41063 non-null  object  
 5   sourceName   41063 non-null  object  
 6   createdDate  41063 non-null  object  
 7   closedDate   41063 non-null  object  
 8   address      41063 non-null  object  
 9   latitude     41063 non-null  float64 
 10  longitude    41063 non-null  float64 
 11  AreaName     41063 non-null  object  
 12  DATE OCC     41063 non-null  object  
 13  TIME OCC     41063 non-null  float64 
 14  Rpt Dist No  41063 non-null  float64 
 15  Crm Cd       41063 non-null  float64 
 16  Crm Cd Desc  41063 non-null  object  
 17  Status Desc  41063 non-null  object  
 18  LOCATION     41063 non-nul

### Merging 2 dataframes with only 1000 rows results in a dataframe wit 41063 rows. How is this even possible??? 

In [ ]:
df311_subset.to_csv('df311_subset.csv')

In [ ]:
dfcrime.to_csv('dfcrime.csv')

In [38]:
df311_40 = df311_subset.head(40)
dfcrime_40 = dfcrime.head(40)

In [39]:
df2019_40 = pd.merge(df311_40, dfcrime_40, left_on = 'AreaName', right_on = 'AreaName', how = 'inner', indicator = True)

In [40]:
df2019_40

,srnumber,councilId,councilName,typeName,agencyName,sourceName,createdDate,closedDate,address,latitude,longitude,AreaName,DATE OCC,TIME OCC,Rpt Dist No,Crm Cd,Crm Cd Desc,Status Desc,LOCATION,_merge
0,1-1262692791,32.0,Greater Wilshire,Bulky Items,Sanitation Bureau,Website,2019-01-01T00:02:00,2019-01-04T11:03:00,"616 N GRAMERCY PL, 90004",34.082258,-118.312461,OLYMPIC,7/16/2019,1500.0,2014.0,354.0,THEFT OF IDENTITY,Invest Cont,200 S HARVARD BL,both
1,1-1262698261,32.0,Greater Wilshire,Bulky Items,Sanitation Bureau,Mobile App,2019-01-01T03:35:00,2019-01-05T12:36:00,"3906 W 8TH ST, 90005",34.057891,-118.311685,OLYMPIC,7/16/2019,1500.0,2014.0,354.0,THEFT OF IDENTITY,Invest Cont,200 S HARVARD BL,both
2,1-1262698294,32.0,Greater Wilshire,Bulky Items,Sanitation Bureau,Mobile App,2019-01-01T03:37:00,2019-01-05T12:37:00,"802 S ST ANDREWS PL, 90005",34.057782,-118.311077,OLYMPIC,7/16/2019,1500.0,2014.0,354.0,THEFT OF IDENTITY,Invest Cont,200 S HARVARD BL,both
3,1-1262693571,60.0,Northridge West,Graffiti,Office of Community Beautification,Website,2019-01-01T00:10:00,2019-01-03T00:27:00,"9167 N RESEDA BLVD, 91324",34.236888,-118.536385,DEVONSHIRE,5/29/2019,730.0,1798.0,901.0,VIOLATION OF RESTRAINING ORDER,Adult Other,8900 AQUEDUCT AV,both
4,1-1262693571,60.0,Northridge West,Graffiti,Office of Community Beautification,Website,2019-01-01T00:10:00,2019-01-03T00:27:00,"9167 N RESEDA BLVD, 91324",34.236888,-118.536385,DEVONSHIRE,2/16/2019,1700.0,1745.0,740.0,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",Invest Cont,10300 ENCINO AV,both
5,1-1262692831,59.0,Northridge South,Illegal Dumping,Sanitation Bureau,Website,2019-01-01T00:19:00,2019-01-07T09:39:00,"8752 N YOLANDA AVE, 91324",34.230222,-118.539758,DEVONSHIRE,5/29/2019,730.0,1798.0,901.0,VIOLATION OF RESTRAINING ORDER,Adult Other,8900 AQUEDUCT AV,both
6,1-1262692831,59.0,Northridge South,Illegal Dumping,Sanitation Bureau,Website,2019-01-01T00:19:00,2019-01-07T09:39:00,"8752 N YOLANDA AVE, 91324",34.230222,-118.539758,DEVONSHIRE,2/16/2019,1700.0,1745.0,740.0,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",Invest Cont,10300 ENCINO AV,both
7,1-1262692061,59.0,Northridge South,Graffiti,Office of Community Beautification,Website,2019-01-01T00:28:00,2019-01-03T00:28:00,"9055 N RESEDA BLVD, 91324",34.235078,-118.536392,DEVONSHIRE,5/29/2019,730.0,1798.0,901.0,VIOLATION OF RESTRAINING ORDER,Adult Other,8900 AQUEDUCT AV,both
8,1-1262692061,59.0,Northridge South,Graffiti,Office of Community Beautification,Website,2019-01-01T00:28:00,2019-01-03T00:28:00,"9055 N RESEDA BLVD, 91324",34.235078,-118.536392,DEVONSHIRE,2/16/2019,1700.0,1745.0,740.0,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",Invest Cont,10300 ENCINO AV,both
9,1-1262694421,58.0,Northridge East,Graffiti,Office of Community Beautification,Website,2019-01-01T00:32:00,2019-01-03T16:27:00,"9312 N ZELZAH AVE, 91325",34.239704,-118.523064,DEVONSHIRE,5/29/2019,730.0,1798.0,901.0,VIOLATION OF RESTRAINING ORDER,Adult Other,8900 AQUEDUCT AV,both


### Checkout rows with OLYMPIC areaname, the crime data is just repeated 3 times for each entry in 311 dataframe. So the same thing will be happening for all other multiple entries and hence the memory os getting bloated on the final merge. 

In [41]:
df2019_40.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   srnumber     50 non-null     object  
 1   councilId    50 non-null     float64 
 2   councilName  50 non-null     object  
 3   typeName     50 non-null     object  
 4   agencyName   50 non-null     object  
 5   sourceName   50 non-null     object  
 6   createdDate  50 non-null     object  
 7   closedDate   50 non-null     object  
 8   address      50 non-null     object  
 9   latitude     50 non-null     float64 
 10  longitude    50 non-null     float64 
 11  AreaName     50 non-null     object  
 12  DATE OCC     50 non-null     object  
 13  TIME OCC     50 non-null     float64 
 14  Rpt Dist No  50 non-null     float64 
 15  Crm Cd       50 non-null     float64 
 16  Crm Cd Desc  50 non-null     object  
 17  Status Desc  50 non-null     object  
 18  LOCATION     50 non-null     obj

### Going to try join now.

In [42]:
dfcrime_1000 = dfcrime_1000.set_index('AreaName')

In [43]:
df311_1000 = df311_1000.set_index('AreaName')

In [44]:
df1000 = dfcrime_1000.join(df311_1000, how = 'inner')

In [45]:
df1000.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41063 entries, 77TH STREET to WILSHIRE
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DATE OCC     41063 non-null  object 
 1   TIME OCC     41063 non-null  float64
 2   Rpt Dist No  41063 non-null  float64
 3   Crm Cd       41063 non-null  float64
 4   Crm Cd Desc  41063 non-null  object 
 5   Status Desc  41063 non-null  object 
 6   LOCATION     41063 non-null  object 
 7   srnumber     41063 non-null  object 
 8   councilId    41063 non-null  float64
 9   councilName  41063 non-null  object 
 10  typeName     41063 non-null  object 
 11  agencyName   41063 non-null  object 
 12  sourceName   41063 non-null  object 
 13  createdDate  41063 non-null  object 
 14  closedDate   41063 non-null  object 
 15  address      41063 non-null  object 
 16  latitude     41063 non-null  float64
 17  longitude    41063 non-null  float64
dtypes: float64(6), object(12)
memory usage

### Still the same output as the one from merge.